# DATA CARD

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [40]:
df1 = pd.read_sql_query("select w.name warehouse, DATENAME(weekday, '2022-04-18') as WeekDayName, oc.name operation, count(distinct(t.id)) quantity\
    from datex_footprint.Tasks t\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
    where t.createdSysDateTime > '2022-04-18' AND t.createdSysDateTime < '2022-04-19' AND oc.name in ('Batch Move', 'Count', 'Inventory Adjustment', 'InventoryMove', 'License Plate Pick', 'LicensePlateMove', 'Picking', 'Receiving')\
    group by w.name, oc.name\
    order by warehouse, operation", cnxn)

In [37]:
df2 = pd.read_sql_query("select w.name warehouse, DATENAME(weekday, '2022-04-19') as WeekDayName, oc.name operation, count(distinct(t.id)) quantity\
    from datex_footprint.Tasks t\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
    where t.createdSysDateTime > '2022-04-19' AND t.createdSysDateTime < '2022-04-20' AND oc.name in ('Batch Move', 'Count', 'Inventory Adjustment', 'InventoryMove', 'License Plate Pick', 'LicensePlateMove', 'Picking', 'Receiving')\
    group by w.name, oc.name\
    order by warehouse, operation", cnxn)

In [38]:
df3 = pd.read_sql_query("select w.name warehouse, DATENAME(weekday, getdate()) as WeekDayName, oc.name operation, count(distinct(t.id)) quantity\
    from datex_footprint.Tasks t\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
    where t.createdSysDateTime > '2022-04-20' AND t.createdSysDateTime < '2022-04-21' AND oc.name in ('Batch Move', 'Count', 'Inventory Adjustment', 'InventoryMove', 'License Plate Pick', 'LicensePlateMove', 'Picking', 'Receiving')\
    group by w.name, oc.name\
    order by warehouse, operation", cnxn)

# Create Data Card

In [47]:
df = df1.append(df2).append(df3)
df.head()

,warehouse,WeekDayName,operation,quantity
0,10,Monday,Batch Move,251
1,10,Monday,Count,58
2,10,Monday,Inventory Adjustment,11
3,10,Monday,InventoryMove,7
4,10,Monday,License Plate Pick,11


In [49]:
# Convert Statuses to Columns
dc_df = df.pivot(index=['warehouse', 'WeekDayName'], columns='operation', values='quantity')
dc_df

operation              Batch Move  Count  Inventory Adjustment  InventoryMove  \
warehouse WeekDayName                                                           
10        Monday            251.0   58.0                  11.0            7.0   
          Tuesday            98.0  136.0                  11.0            7.0   
          Wednesday         122.0   21.0                   4.0            7.0   
102       Monday              NaN    NaN                   NaN            NaN   
          Tuesday             NaN    NaN                   NaN            NaN   
          Wednesday           NaN    NaN                   NaN            NaN   
12        Monday              NaN    NaN                   NaN            NaN   
          Tuesday             NaN    NaN                   1.0            NaN   
          Wednesday           NaN    NaN                   NaN            NaN   
15        Monday              NaN    7.0                   NaN            1.0   
          Tuesday             NaN   57.0                   1.0            NaN   
          Wednesday           NaN   18.0                   1.0            NaN   
16        Monday              NaN    NaN                   NaN            1.0   
          Tuesday             NaN    NaN                   NaN            NaN   
          Wednesday           NaN    NaN                   NaN            NaN   
17        Monday              NaN    NaN                   NaN            NaN   
18        Monday            166.0    NaN                   1.0            1.0   
          Tuesday            25.0    NaN                   NaN            NaN   
          Wednesday           8.0    NaN                   NaN            2.0   
19        Monday              NaN    NaN                   NaN            NaN   
          Tuesday             NaN    5.0                   NaN           66.0   
          Wednesday           NaN    2.0                   1.0            9.0   
20        Monday              NaN   73.0                   2.0            2.0   
          Tuesday             NaN   80.0                   2.0            NaN   
          Wednesday           NaN   63.0                   1.0            1.0   
21        Monday              NaN    NaN                   NaN            NaN   
          Tuesday             NaN    NaN                   NaN            NaN   
          Wednesday           NaN    NaN                   NaN            NaN   
22        Monday              NaN    NaN                   NaN            NaN   
          Tuesday             NaN    NaN                  15.0            NaN   
          Wednesday           NaN    NaN                   NaN            NaN   

operation              License Plate Pick  LicensePlateMove  Picking  \
warehouse WeekDayName                                                  
10        Monday                     11.0              27.0   2453.0   
          Tuesday                    21.0              54.0   1016.0   
          Wednesday                  45.0              56.0   1163.0   
102       Monday                      1.0               1.0      1.0   
          Tuesday                     1.0               1.0      1.0   
          Wednesday                   3.0               3.0      3.0   
12        Monday                      NaN              67.0      NaN   
          Tuesday                    24.0               6.0     34.0   
          Wednesday                   NaN               8.0    140.0   
15        Monday                      NaN               5.0    316.0   
          Tuesday                     3.0               2.0    129.0   
          Wednesday                   NaN               1.0     70.0   
16        Monday                     27.0             105.0     48.0   
          Tuesday                     NaN              36.0     72.0   
          Wednesday                  73.0              17.0     67.0   
17        Monday                      NaN               NaN      NaN   
18        Monday              

In [50]:
# Convert NaN to 0
dc_df_zero = dc_df.fillna(0)
# Convert Float to Int
df2 = dc_df_zero.astype(int)
# Replace Zero with Empty
df2.replace([0, ''], '', inplace=True)

df2

operation             Batch Move Count Inventory Adjustment InventoryMove  \
warehouse WeekDayName                                                       
10        Monday             251    58                   11             7   
          Tuesday             98   136                   11             7   
          Wednesday          122    21                    4             7   
102       Monday                                                            
          Tuesday                                                           
          Wednesday                                                         
12        Monday                                                            
          Tuesday                                         1                 
          Wednesday                                                         
15        Monday                     7                                  1   
          Tuesday                   57                    1                 
          Wednesday                 18                    1                 
16        Monday                                                        1   
          Tuesday                                                           
          Wednesday                                                         
17        Monday                                                            
18        Monday             166                          1             1   
          Tuesday             25                                            
          Wednesday            8                                        2   
19        Monday                                                            
          Tuesday                    5                                 66   
          Wednesday                  2                    1             9   
20        Monday                    73                    2             2   
          Tuesday                   80                    2                 
          Wednesday                 63                    1             1   
21        Monday                                                            
          Tuesday                                                           
          Wednesday                                                         
22        Monday                                                            
          Tuesday                                        15                 
          Wednesday                                                         

operation             License Plate Pick LicensePlateMove Picking Receiving  
warehouse WeekDayName                                                        
10        Monday                      11               27    2453       189  
          Tuesday                     21               54    1016        98  
          Wednesday                   45               56    1163       137  
102       Monday                       1                1       1            
          Tuesday                      1                1       1            
          Wednesday                    3                3       3            
12        Monday                                       67                61  
          Tuesday                     24                6      34         6  
          Wednesday                                     8     140            
15        Monday                                        5     316         2  
          Tuesday                      3                2     129            
          Wednesday                                     1      70         2  
16        Monday                      27              105      48        71  
          Tuesday                                      36      72        83  
          Wednesday                   73               17      67        24  
17        Monday                                                         86  
18        Monday                                      

In [55]:
# Save Waves DF into json file
df2.to_json('data/card.json')

In [56]:
# Save Waves DF into json file
df2.to_json('data/card_records.json', orient='records')

In [57]:
# Save Waves DF into json file
df2.to_json('data/card_split.json', orient='split')

In [58]:
# Save Waves DF into json file
df2.to_json('data/card_index.json', orient='index')

In [59]:
# Save Waves DF into json file
df2.to_json('data/card_columns.json', orient='columns')

In [60]:
# Save Waves DF into json file
df2.to_json('data/card_values.json', orient='values')

In [61]:
# Save Waves DF into json file
df2.to_json('data/card_table.json', orient='table')

In [ ]:
df3 df2.melt

In [52]:
# Close connection
cnxn.close()